# Separate the scans

We scanned multiple fish in [a special sample holder](https://github.com/TomoGraphics/Hol3Drs/blob/master/STL/Stickleback.Multiple.stl).
This notebook is used to separate them into different bunch of reonstructions.

The cells below are used to set up the whole notebook.
They load needed libraries and set some default values.

In [ ]:
# Load the modules we need
import platform
import os
import glob
import pandas
import imageio
import numpy
import matplotlib.pyplot as plt
from matplotlib_scalebar.scalebar import ScaleBar
import seaborn
import dask
import dask_image.imread
from dask.distributed import Client, LocalCluster
import skimage
from tqdm.auto import tqdm, trange

In [ ]:
# Load our own log file parsing code
# This is loaded as a submodule to alleviate excessive copy-pasting between *all* projects we do
# See https://github.com/habi/BrukerSkyScanLogfileRuminator for details on its inner workings
from BrukerSkyScanLogfileRuminator.parsing_functions import *

In [ ]:
# Set dask temporary folder
# Do this before creating a client: https://stackoverflow.com/a/62804525/323100
# We use the fast internal SSD for speed reasons
import tempfile
if 'Linux' in platform.system():
    # Check if me mounted the FastSSD, otherwise go to standard tmp file
    if os.path.exists(os.path.join(os.sep, 'media', 'habi', 'Fast_SSD')):
        tmp = os.path.join(os.sep, 'media', 'habi', 'Fast_SSD', 'tmp')
    else:
        tmp = tempfile.gettempdir()
elif 'Darwin' in platform.system():
    tmp = tempfile.gettempdir()
else:
    if 'anaklin' in platform.node():
        tmp = os.path.join('F:\\tmp')
    else:
        tmp = os.path.join('D:\\tmp')
dask.config.set({'temporary_directory': tmp})
print('Dask temporary files go to %s' % dask.config.get('temporary_directory'))

In [ ]:
from dask.distributed import Client
client = Client()

In [ ]:
client

In [ ]:
seaborn.set_context('notebook')

In [ ]:
# Set up figure defaults
plt.rc('image', cmap='gray', interpolation='nearest')  # Display all images in b&w and with 'nearest' interpolation
# plt.rcParams['figure.figsize'] = (16, 9)  # Size up figures a bit
plt.rcParams['figure.dpi'] = 200

In [ ]:
# Setup scale bar defaults
plt.rcParams['scalebar.location'] = 'lower right'
plt.rcParams['scalebar.frameon'] = False
plt.rcParams['scalebar.color'] = 'white'

In [ ]:
# Display all plots identically
lines = 3
# And then do something like
# plt.subplot(lines, int(numpy.ceil(len(Data) / float(lines))), c + 1)

Since the (tomographic) data can reside on different drives we set a folder to use below

In [ ]:
# Different locations if running either on Linux or Windows
FastSSD = True
if 'Linux' in platform.system():
    if FastSSD:
        BasePath = os.path.join(os.sep, 'media', 'habi', 'Fast_SSD')
    else:
        BasePath = os.path.join(os.path.sep, 'home', 'habi', '2214')
elif 'Windows' in platform.system():
    if FastSSD:
        BasePath = os.path.join('F:\\')
    else:
        BasePath = os.path.join('N:\\')
Root = os.path.join(BasePath, 'IEE Stickleback')
print('We are loading all the data from %s' % Root)

Now that we are set up, actually start to load/ingest the data.

In [ ]:
# Make us a dataframe for saving all that we need
Data = pandas.DataFrame()

In [ ]:
# Get *all* log files
# Using os.walk is way faster than using recursive glob.glob, see DataWrangling.ipynb for details
# Not sorting the found logfiles is also making it quicker
Data['LogFile'] = [os.path.join(root, name)
                   for root, dirs, files in os.walk(Root)
                   for name in files
                   if name.endswith((".log"))]

In [ ]:
# Get all folders
Data['Folder'] = [os.path.dirname(f) for f in Data['LogFile']]

In [ ]:
# Show a (small) sampler of the loaded data as a first check
Data.sample(n=5)

In [ ]:
# Get rid of all the logfiles from all the folders that might be on disk but that we don't want to load the data from
for c, row in Data.iterrows():
    # Since this notebook only deals with the 'BucketOfFish' scans, drop all others
    if 'Bucket' not in row.Folder:
        Data.drop([c], inplace=True)    
    elif 'rec' not in row.Folder:
        Data.drop([c], inplace=True)
    elif 'SubScan' in row.Folder:
        Data.drop([c], inplace=True)
    elif 'rectmp.log' in row.LogFile:
        Data.drop([c], inplace=True)
# Reset dataframe to something that we would get if we only would have loaded the 'rec' files
Data = Data.reset_index(drop=True)

In [ ]:
# Generate us some meaningful colums in the dataframe
Data['Sample'] = [os.path.basename(l).replace('_rec.log','') for l in Data['LogFile']]
Data['Scan'] = [os.path.basename(os.path.dirname(l)) for l in Data['LogFile']]

In [ ]:
# Quickly show the data from the last loaded scans
Data.tail()

In [ ]:
# Load the file names of all the reconstructions of all the scans
Data['Filenames Reconstructions'] = [glob.glob(os.path.join(f, '*rec0*.png')) for f in Data['Folder']]
# How many reconstructions do we have?
Data['Number of reconstructions'] = [len(r) for r in Data['Filenames Reconstructions']]

In [ ]:
# Drop samples which have either not been reconstructed yet or of which we deleted the reconstructions with
Data = Data[Data['Number of reconstructions'] > 0]
# Reset the dataframe count/index for easier indexing afterwards
Data.reset_index(drop=True, inplace=True)
print('We have %s folders with reconstructions' % (len(Data)))

In [ ]:
# Get parameters to doublecheck from logfiles
Data['Voxelsize'] = [pixelsize(log) for log in Data['LogFile']]
Data['Filter'] = [whichfilter(log) for log in Data['LogFile']]
Data['Exposuretime'] = [exposuretime(log) for log in Data['LogFile']]
Data['Scanner'] = [scanner(log) for log in Data['LogFile']]
Data['Averaging'] = [averaging(log) for log in Data['LogFile']]
Data['ProjectionSize'] = [projection_size(log) for log in Data['LogFile']]
Data['RotationStep'] = [rotationstep(log) for log in Data['LogFile']]
Data['Grayvalue'] = [reconstruction_grayvalue(log) for log in Data['LogFile']]
Data['RingartefactCorrection'] = [ringremoval(log) for log in Data['LogFile']]
Data['BeamHardeningCorrection'] = [beamhardening(log) for log in Data['LogFile']]
Data['DefectPixelMasking'] = [defectpixelmasking(log) for log in Data['LogFile']]
Data['Scan date'] = [scandate(log) for log in Data['LogFile']]

In [ ]:
# # Load all reconstructions DASK arrays
# Reconstructions = [dask_image.imread.imread(os.path.join(folder,'*rec*.png')) for folder in Data['Folder']]
# Load all reconstructions into ephemereal DASK arrays, with a nice progress bar...
Reconstructions = [None] * len(Data)
for c, row in tqdm(Data.iterrows(),
                   desc='Loading reconstructions',
                   total=len(Data)):
    Reconstructions[c] = dask_image.imread.imread(os.path.join(row['Folder'], '*rec*.png'))

In [ ]:
# Show and labbook image, if present
Data['Bucket'] = [(l).split(os.sep)[-3].split('_')[-1] for l in Data['LogFile']]

In [ ]:
Data['LabbookImage'] = [dask_image.imread.imread(os.path.join(os.path.dirname(f), '_labbook.jpg')).squeeze() for f in Data['Folder']]
Data['OverviewImage'] = [dask_image.imread.imread(os.path.join(os.path.dirname(f), '_overview.jpg')).squeeze()
                         if os.path.exists(os.path.join(os.path.dirname(f), '_overview.jpg'))
                         else numpy.random.random((2**8,2**8)) for f in Data['Folder']]

In [ ]:
# The three cardinal directions
# Names adapted to fishes: https://en.wikipedia.org/wiki/Fish_anatomy#Body
directions = ['Anteroposterior',
              'Lateral',
              'Dorsoventral']

In [ ]:
# Read or calculate the directional MIPs, put them into the dataframe and save them to disk
for d, direction in enumerate(directions):
    Data['MIP_' + direction] = ''
for c, row in tqdm(Data.iterrows(), desc='Working on MIPs', total=len(Data)):
    for d, direction in tqdm(enumerate(directions),
                             desc='%s/%s' % (row['Sample'], row['Scan']),
                             leave=False,
                             total=len(directions)):
        outfilepath = os.path.join(os.path.dirname(row['Folder']),
                                   '%s.%s.MIP.%s.png' % (row['Sample'], row['Scan'], direction))
        if os.path.exists(outfilepath):
            try:
                Data.at[c, 'MIP_' + direction] = dask_image.imread.imread(outfilepath).squeeze()
            except:
                print('Something went wrong with the %s image of %s/%s %s' % (direction, row['Sample'], row['Scan']))
                print('Try to delete the %s and restart the notebook' % outfilepath)
        else:
            # Generate MIP
            Data.at[c, 'MIP_' + direction] = Reconstructions[c].max(axis=d).squeeze().compute()
            # Save it out
            imageio.imwrite(outfilepath, Data.at[c, 'MIP_' + direction].astype('uint8'))

In [ ]:
def getLargestCC(segmentation):
    # Based on https://stackoverflow.com/a/55110923
    labels = skimage.measure.label(segmentation)
    assert( labels.max() != 0 ) # assume at least 1 CC
    largestCC = labels == numpy.argmax(numpy.bincount(labels.flat)[1:])+1
    return largestCC

In [ ]:
import matplotlib

In [ ]:
def vial_label_extractor(whichscan, threshold=None, part=300, verbose=True):
    # Let's get out the numbers, they are 'hidden' in the lower part
    bottom_mip = Reconstructions[whichscan][:300].max(axis=0)[:,:,0].compute()
    if not threshold:
        threshold = skimage.filters.threshold_otsu(bottom_mip[bottom_mip>10])
    # remove largest component from thresholded bottom MIP
    # The largest component are the separation walls of the bucket
    numbers = numpy.bitwise_xor(bottom_mip>threshold, getLargestCC(bottom_mip>threshold))
    # Clean up the image by removing small objects
    numbers_cleaned = skimage.morphology.remove_small_objects(numbers, min_size=10000)
    # only labels should remain
    if verbose:
        plt.subplot(121)
        plt.imshow(Data['MIP_Dorsoventral'][whichscan])
        plt.gca().add_artist(ScaleBar(Data['Voxelsize'][whichscan], 'um'))
        plt.gca().add_artist(matplotlib.patches.Rectangle((0, 0), Data['MIP_Dorsoventral'][whichscan].shape[1], part, alpha=0.618))
        plt.title('Bucket %s' % Data['Bucket'][whichscan])
        plt.axis('off')
        plt.subplot(122)
        plt.imshow(numbers_cleaned)
        plt.title('MIP of marked region\n%s recs >%s - their largest CC' % (part, threshold))
        plt.gca().add_artist(ScaleBar(Data['Voxelsize'][whichscan], 'um'))
        plt.axis('off')
        plt.savefig('%s.%s.Labels.Extracted.png' % (os.path.join(os.path.dirname(Data['Folder'][whichscan]), Data.Sample[whichscan]), Data.Scan[whichscan]))
        plt.show()
    return(numbers_cleaned)

In [ ]:
Data['VialLabels'] = [vial_label_extractor(i, verbose=True) for i in range(len(Data))]

In [ ]:
def detect_fish_position(whichscan, threshold=None, verbose=False):
    import matplotlib.patches
    # Detect the fish positions based on blobs in the top-down MIP
    regions = None
    td_mip = Data.MIP_Anteroposterior[whichscan][:,:,0].compute()
    if not threshold:
        threshold = skimage.filters.threshold_otsu(td_mip[td_mip>10])
    td_mip_thresholded = td_mip>threshold
    # Clean speckles, assuming all fish are larger than 1000 px
    cleaned = skimage.morphology.remove_small_objects(td_mip_thresholded,
                                                      min_size=1000)
    # Label image...
    label_image = skimage.measure.label(cleaned)
    # Detect regions
    regions = skimage.measure.regionprops(label_image)
    if verbose:
        plt.subplot(121)
        plt.imshow(td_mip)
        plt.imshow(Data['VialLabels'][whichscan], alpha=0.5)    
        plt.title('Bucket %s' % Data['Bucket'][whichscan])
        plt.gca().add_artist(ScaleBar(Data['Voxelsize'][whichscan], 'um'))
        plt.axis('off')
        plt.subplot(122)
        # to make the background transparent, pass the value of `bg_label`,
        # and leave `bg_color` as `None` and `kind` as `overlay`   
        plt.imshow(skimage.color.label2rgb(label_image, image=td_mip, bg_label=0))
        for c, region in enumerate(regions):
            # draw rectangle around segmented fish
            minr, minc, maxr, maxc = region.bbox
            rect = matplotlib.patches.Rectangle((minc, minr), maxc - minc, maxr - minr,
                                      fill=False, edgecolor='white', ls='--')
            plt.plot(region.centroid)
            plt.gca().add_patch(rect)
            plt.annotate('%s' % region.label, xy=((minc + maxc) / 2, minr - 15), color='white', ha='center')
        plt.title('Detected fish')
        plt.gca().add_artist(ScaleBar(Data['Voxelsize'][whichscan], 'um'))
        plt.axis('off')
        plt.savefig('%s.%s.Labels.Detected.png' % (os.path.join(os.path.dirname(Data['Folder'][whichscan]), Data.Sample[whichscan]), Data.Scan[whichscan]))        
        plt.show()
    return(regions)

In [ ]:
Data['Regions'] = [detect_fish_position(i) for i in range(len(Data))]

In [ ]:
# Construct us a fish number
Data['FishNumber'] = [[reg.label + (len(region) * c - 1) for reg in region] for c, region in enumerate(Data.Regions)]

In [ ]:
# Overwrite blunders in first batches with their correct numbers
# Bucket A
Data['FishNumber'][0] = [fn + 1 for fn in range(len(Data['FishNumber'][0]))]
Data['FishNumber'][0][2] = 30
# Bucket B
Data['FishNumber'][1][0] = 3

In [ ]:
# Construct us a fish name
Data['FishID'] = [['FG.X23.%03d' % n for n in number] for number in Data.FishNumber]

In [ ]:
def reorder_list(list, neworder = [0, 2, 4, 5, 3, 1]):
    '''
    Shuffle a list to a defined order
    We *deliberately* want a new list, so we can keep the old one
    https://stackoverflow.com/questions/2177590/how-can-i-reorder-a-list#comment106984501_2177607
    '''
    ordered_list = [list[i] for i in neworder]
    return(ordered_list)

In [ ]:
# We had a *very* hard time to correctly shuffle the list
# If we both print out the label from the image and the shuffeled list, it's easier to check :)
print('from image', [i+1 for i in range(6)])
# For most buckets
print('from label', reorder_list([i+1 for i in range(6)]))
# For bucket C
print('from label', reorder_list([i+1 for i in range(6)], neworder = [0, 2, 3, 5, 4, 1]))

In [ ]:
# Reorder regions
# Since we always reconstructed vial 1 up north, the order is consistent (most of the time)
# We can thus just reshuffle *all* the detected regions for writing them out correctly afterwards
Data['Regions_Ordered'] = [reorder_list(regions) for regions in Data['Regions']]
# WARNING the region.labels do NOT correspond to the label anymore :)

In [ ]:
# If the reshuffling above does not work for one scan, we can reorder it deliberately again.
# Bucket C: label 4 and 5 are swapped, due to one having a bigger fish than the others
Data['Regions_Ordered'][2] = reorder_list(Data['Regions'][2], neworder = [0, 2, 3, 5, 4, 1])

In [ ]:
def doublecheck_fish_position(whichscan):
    plt.subplot(121)
    plt.imshow(Data['MIP_Anteroposterior'][whichscan])
    plt.imshow(numpy.ma.masked_equal(Data['VialLabels'][whichscan], False), cmap='viridis_r')
    for c, region in enumerate(Data['Regions'][whichscan]):
        plt.annotate('%s' % str(c+1),
                     xy=(region.centroid[1] + 100, region.centroid[0]),
                     color='black',
                     va='center',
                     bbox=dict(fc="white", alpha=0.618))
    plt.title('Bucket %s, calculated labels' % Data['Bucket'][whichscan])
    plt.gca().add_artist(ScaleBar(Data['Voxelsize'][whichscan], 'um'))
    plt.axis('off')
    plt.subplot(122)
    plt.imshow(Data['MIP_Anteroposterior'][whichscan])
    plt.imshow(numpy.ma.masked_equal(Data['VialLabels'][whichscan], False), cmap='viridis_r')
    for c, region in enumerate(Data['Regions_Ordered'][whichscan]):
        plt.annotate('%s:%s' % (c+1, Data['FishID'][whichscan][c]),                     
                     xy=(region.centroid[1], region.centroid[0]),
                     color='black',
                     fontsize=8,
                     bbox=dict(fc="white", alpha=0.618))
    plt.title('Resorted label:mapped IDs')
    plt.gca().add_artist(ScaleBar(Data['Voxelsize'][whichscan], 'um'))
    plt.axis('off')
    plt.savefig('%s.%s.Labels.Sorted.png' % (os.path.join(os.path.dirname(Data['Folder'][whichscan]), Data.Sample[whichscan]), Data.Scan[whichscan]))
    plt.show()
    return()

In [ ]:
for i in range(len(Data)):
    doublecheck_fish_position(i)

In [ ]:
def regionextractor(whichscan, buffer=50):
    os.makedirs(Data.Folder[whichscan] + '_regions', exist_ok=True)
    for c, region in tqdm(enumerate(Data['Regions_Ordered'][whichscan]),
                          total=len(Data['Regions_Ordered'][whichscan])):
        # plt.subplot(6, 2, (2 * c ) +1)
        plt.subplot(2,6,c+1)
        plt.imshow(Data['MIP_Anteroposterior'][whichscan][region.bbox[0]-buffer:region.bbox[2]+buffer,region.bbox[1]-buffer:region.bbox[3]+buffer])
        plt.imshow(numpy.ma.masked_equal(Data['VialLabels'][whichscan][region.bbox[0]-buffer:region.bbox[2]+buffer,region.bbox[1]-buffer:region.bbox[3]+buffer],
                                         False),
                   cmap='viridis_r')
        plt.gca().add_artist(ScaleBar(Data['Voxelsize'][whichscan], 'um'))
        plt.title('%s:Cut original' % str(c + 1))
        plt.axis('off')
        # plt.subplot(6, 2, (2 * c ) + 2)
        plt.subplot(2,6,c+1+6)
        outputfilename = os.path.join(Data.Folder[whichscan] + '_regions', 'region_%s_%s.zarr' % (str(c+1),
                                                                                                  Data['FishID'][whichscan][c]))
        if not os.path.exists(outputfilename):
            currentregion = Reconstructions[whichscan][:,region.bbox[0]-buffer:region.bbox[2]+buffer,region.bbox[1]-buffer:region.bbox[3]+buffer]
            print('writing to', outputfilename[len(Root)+1:])
            dask.array.to_zarr(currentregion.rechunk('auto'), outputfilename)
        else:
            print(outputfilename[len(Root)+1:], 'already exists')
            currentregion = dask.array.from_zarr(outputfilename)
        # Recalculate MIP for double-checking
        plt.imshow(currentregion.max(axis=0))
        plt.gca().add_artist(ScaleBar(Data['Voxelsize'][whichscan], 'um'))
        plt.title(Data['FishID'][whichscan][c])
        plt.axis('off')
    # plt.tight_layout()
    plt.savefig('%s.%s.Regions.png' % (os.path.join(os.path.dirname(Data['Folder'][whichscan] + '_regions'), Data.Sample[0]), Data.Scan[0]))
    plt.show()
    return()

In [ ]:
# Explicitly state the buffer, we need it later for the log file of the regions
buffer = 50
for i in range(len(Data)):
    regionextractor(i, buffer=buffer)

In [ ]:
# Show all information we have, to double-check the mapping
for c, row in Data.iterrows():
    plt.subplot(131)
    plt.imshow(row.LabbookImage)
    plt.title('Bucket %s: Labbook' % row.Bucket)
    plt.axis('off')
    plt.subplot(132)
    plt.imshow(row.OverviewImage)
    plt.title('Bucket %s: Tubes' % row.Bucket)
    plt.axis('off')
    plt.subplot(133)
    plt.imshow(Data['MIP_Anteroposterior'][c])
    plt.imshow(numpy.ma.masked_equal(Data['VialLabels'][c], False), cmap='viridis_r')
    for d, region in enumerate(Data['Regions_Ordered'][c]):
        plt.annotate('%s:%s' % (d+1, Data['FishID'][c][d]),                     
                     xy=(region.centroid[1], region.centroid[0]),
                     color='black',
                     ha='center',
                     va='center',
                     fontsize=8,
                     bbox=dict(fc="white", alpha=0.618))
    plt.title('Bucket %s: IDs' % row.Bucket)
    plt.gca().add_artist(ScaleBar(Data['Voxelsize'][c], 'um'))
    plt.axis('off')
    plt.savefig('%s.%s.Mapping.png' % (os.path.join(os.path.dirname(row['Folder']), row.Sample), row.Scan))
    plt.show()

In [ ]:
# Read in regions again
RegionZarrFiles = [glob.glob(os.path.join(folder + '_regions', '*.zarr')) for folder in Data['Folder']]
Regions = [[dask.array.from_zarr(f) for f in files] for files in RegionZarrFiles]

In [ ]:
# We want to generate log files for the cutout regions
# Aeons ago, we wrote a little wrapper function to log stuff at TOMCAT
# https://github.com/habi/TOMCAT/blob/master/postscan/StackedScanOverlapFinder.py#L104
# The function below is slightly tweaked from there
def myLogger(logfilename, verbose=False):
    import logging
    logger = logging.getLogger(logfilename)
    logger.setLevel(logging.INFO)
    handler = logging.FileHandler(logfilename, 'w')
    logger.addHandler(handler)
    if verbose:
        print('Logging to %s' % logfilename)
    return logger
# Then write to the file with
# logfile = myLogger(Filename))
# logfile.info('Put this into the log file')

In [ ]:
# Save out a log file
for c, row in tqdm(Data.iterrows(), total=len(Data), desc='Writing log files for regions'):
    for d, region in tqdm(enumerate(row.Regions),
                            total=len(row.Regions),
                            desc=Data.Folder[c][len(Root)+1:]):
        # Generate output directory
        os.makedirs(os.path.join(row.Folder + '_regions', row['FishID'][d]), exist_ok=True)
        # Generate logfile name
        logfilename = os.path.join(row.Folder + '_regions', row['FishID'][d], row['FishID'][d] + '.log')
        # Delete logfile, if it already exists
        if os.path.exists(logfilename):
            os.remove(logfilename)
        logfile = myLogger(logfilename)
        logfile.info('Scan = %s' % os.path.join(row.Sample, row.Scan))
        logfile.info('Voxel size = %s um' % row.Voxelsize)
        logfile.info('ID = %s' % row['FishID'][d])
        logfile.info('Vial = %s' % str(d + 1))
        logfile.info('Centroid (x,y) in the original stack = %s, %s' % (int(round(region.centroid[1])), int(round(region.centroid[0]))))
        logfile.info('Bounding box (x1:x2, y1:y2) of this region in the original stack = %s:%s, %s:%s' % (region.bbox[1]-buffer, region.bbox[3]+buffer,
                                                                                                          region.bbox[0]-buffer, region.bbox[2]+buffer))

In [ ]:
len(Data['Filenames Reconstructions'][c])

In [ ]:
Regions[][c]

In [ ]:
def imsaver(image, filename):
    from joblib import Parallel, delayed
    ''' Function for parallelizing writing out imgages '''
    if not os.path.exists(filename):
        imageio.imwrite(filename, image)

In [ ]:
# Save out PNG slices 
for c, row in tqdm(Data.iterrows(), total=len(Data)):
    for d, zarrfile in tqdm(enumerate(Regions[c]),
                            total=len(Regions[c]),
                            desc=Data.Folder[c][len(Root)+1:]):
        # Make output directory
        outputdir = os.path.join(row.Folder + '_regions', row['FishID'][d])
        os.makedirs(outputdir, exist_ok=True)
        outputfilenames = [os.path.join(outputdir,
                                        os.path.basename(fn)).replace(Data.Sample[c], Data['FishID'][c][d]) for fn in Data['Filenames Reconstructions'][c]]
        # for e, slice in tqdm(enumerate(zarrfile),
        #                      total=len(zarrfile),
        #                      desc='Saving to %s' % os.path.splitext(RegionZarrFiles[c][d])[0][len(Root)+1:],
        #                      leave=False):
        #     outfilepath = outputfilenames[e]
        #     print(outfilepath)
#             if not os.path.exists(outfilepath):
#                 imageio.imwrite(outfilepath, slice[:,:,0].astype('uint8'))


        Parallel(n_jobs=-1)(delayed(imsaver)(zarrfile[slice,:,:,0],outputfilenames[slice]) for slice in range(len(filenames)))

In [ ]:
from joblib import Parallel, delayed

In [ ]:
whichscan = 1
outputfilefilenames = [fn.replace('_rec0', '_rec0regio1') for fn in Data['Filenames Reconstructions'][whichscan]]

In [ ]:
len(filenames)

In [ ]:
Parallel(n_jobs=-1)(delayed(imsaver)(Regions[whichscan][0][slice,:,:,0],filenames[slice]) for slice in range(len(filenames)))

In [ ]:
asdfasdfj ?=?

In [ ]:
# Minimize .zarr files to only fish-extent

In [ ]:
for c, region in enumerate(Regions):
    outfilename = RegionZarrFiles[c].replace('_rec.zarr', '.MIPs.png')
    if not os.path.exists(outfilename):
        for d, direction in enumerate(directions):
            plt.subplot(1, 3 , d+1)
            plt.imshow(region.max(axis=d))
            plt.title('Region %s\n%s MIP' % (c, direction))
            plt.axis('off')
            plt.gca().add_artist(ScaleBar(voxelsize, 'um'))
        plt.savefig(outfilename)
        plt.show()
    else:
        print('MIP already saved to %s' % outfilename)

In [ ]:
# Calculate the histograms of one of the MIPs
# Caveat: dask.da.histogram returns histogram AND bins, making each histogram a 'nested' list of [h, b]
Histograms = [dask.array.histogram(dask.array.array(region),
                                          bins=2**8,
                                          range=[0, 2**8]) for region in Regions]
# Actually compute the data and put only h into the dataframe, so we can use it below.
# Discard the bins
Histograms = [h.compute() for h, b in Histograms]

In [ ]:
Thresholds = [skimage.filters.threshold_otsu(region[:,:,:,0][region[:,:,:,0]>10].compute()) for region in Regions]

In [ ]:
for c, hist in enumerate(Histograms):
    plt.semilogy(hist,
                 c=seaborn.color_palette()[c])
    plt.axvline(Thresholds[c],
                label='R%s: %s' % (c, Thresholds[c]),
                c=seaborn.color_palette()[c])
plt.legend()
plt.show()

In [ ]:
for c, region in enumerate(Regions):
    outfilename = RegionZarrFiles[c].replace('_rec.zarr', '.MIPsasdfasdfa.png')
    region = region[:,:,:,0].compute()
    if not os.path.exists(outfilename):
        for d, direction in enumerate(directions):
            plt.subplot(1, 3 , d+1)
            plt.imshow((region>Thresholds[c]).max(axis=d))
            plt.title('Region %s\n%s MIP' % (c, direction))
            plt.axis('off')
            plt.gca().add_artist(ScaleBar(voxelsize, 'um'))
        # plt.savefig(outfilename)
        plt.show()
    else:
        print('MIP already saved to %s' % outfilename[len(Root)+1:])

In [ ]:
Thresholds

In [ ]:
labels = skimage.morphology.label(Regions[0][:,:,:,0]>Thresholds[0])

In [ ]:
import zarr

In [ ]:
# Label fish and save out as .zarr
os.makedirs(Data.Folder[whichscan] + '_labeled', exist_ok=True)
for c, region in tqdm(enumerate(Regions), total=len(regions)):
    plt.subplot(1, 6, c+1)
    currentregion = skimage.morphology.label(region[:,:,:,0]>Thresholds[c])
    outputfilename = os.path.join(Data.Folder[whichscan] + '_labeled', 'region_%s_rec_labeled.zarr' % str(c+1))
    if not os.path.exists(outputfilename):
        print('writing to', outputfilename)
        zarr_out_3D_convenient = zarr.save(outputfilename, currentregion)
    else:
        print(outputfilename[len(Root)+1:], 'already exists')
    currentmip = currentregion.max(axis=0)
    plt.imshow(currentmip)
    plt.gca().add_artist(ScaleBar(voxelsize, 'um'))
    plt.title('Region %s' % c)
    plt.axis('off')
plt.show()

In [ ]:
# Read in labels again
LabelZarrFiles = glob.glob(os.path.join(Data.Folder[whichscan] + '_labeled', '*.zarr'))
Labels = [dask.array.from_zarr(file) for file in LabelZarrFiles]

In [ ]:
for c, region in enumerate(Labels):
    outfilename = LabelZarrFiles[c].replace('_rec_labeled.zarr', '.MIPs.labeled.png')
    print(outfilename)
    # region = region[:,:,:,0].compute()
    if not os.path.exists(outfilename):
        for d, direction in enumerate(directions):
            plt.subplot(1, 3 , d+1)
            plt.imshow((region).max(axis=d))
            plt.title('Region %s\n%s MIP' % (c, direction))
            plt.axis('off')
            plt.gca().add_artist(ScaleBar(voxelsize, 'um'))
        plt.savefig(outfilename)
        plt.show()
    else:
        print('MIP overview image already saved to %s' % outfilename[len(Root)+1:])

In [ ]:
for c, region in enumerate(Labels):
    outfilename = LabelZarrFiles[c].replace('_rec_labeled.zarr', '.Summed.labeled.png')
    if not os.path.exists(outfilename):
        for d, direction in enumerate(directions):
            plt.subplot(1, 3 , d+1)
            plt.imshow((region).sum(axis=d))
            plt.title('Region %s\n%s Sum' % (c, direction))
            plt.axis('off')
            plt.gca().add_artist(ScaleBar(voxelsize, 'um'))
        plt.savefig(outfilename)
        plt.show()
    else:
        print('Summed image already saved to %s' % outfilename[len(Root)+1:])

In [ ]:
slice = 333
for c, r in enumerate(Regions):
    plt.subplot(2,3,c+1)
    plt.imshow(r[slice])
    # plt.imshow((r[:,:,:,0]>Thresholds[c])[slice], alpha=0.5, cmap='viridis')
    plt.imshow(skimage.morphology.label(r[:,:,:,0][slice]>Thresholds[c]), alpha=0.5, cmap='viridis')
    plt.title('R%s' % c)
    plt.axis('off')
plt.show()

In [ ]:
Labels[0]

In [ ]:
# Save out PNG slices for later use
for c, zarrfile in tqdm(enumerate(Labels),
                        total=len(Labels),
                        desc=Data.Folder[whichscan][len(Root)+1:]):
    # Make output directory
    os.makedirs(os.path.splitext(LabelZarrFiles[c])[0], exist_ok=True)
    for d, slice in tqdm(enumerate(zarrfile),
                         total=len(zarrfile),
                         desc='Saving to %s' % os.path.splitext(LabelZarrFiles[c])[0][len(Root)+1:],
                         leave=False):
        outfilepath = os.path.join(os.path.splitext(LabelZarrFiles[c])[0],
                                   os.path.basename(Data['Filenames Reconstructions'][whichscan][d])).replace('_rec00', '_region_%s_labeled_rec00' % str(c+1))
        if not os.path.exists(outfilepath):
            # plt.imshow(slice.compute())
            # plt.show()
            # print(type(slice))
            imageio.imwrite(outfilepath, slice.compute().astype('uint8'))

In [ ]:
blobs = skimage.feature.blob_dog(clean)

In [ ]:
blobs

In [ ]:
plt.subplot(121)
plt.imshow(clean)
plt.subplot(122)
plt.imshow(mip)